# PRMT-2416 Vision 'Core extract not sent' transfers September 2021

## Context
Vision have been investigating the large issue of transfers failing because the core extract isn’t sent. They want data from September to help them further with their investigation.

## Scope

Produce a spreadsheet of all transfers in September, where Vision is the sender, that resulted in technical failure, core extract not sent, no error code (there should be approx 555). 

### List should have:

* Sending practice name 
* Sending practice ODS
* Sending practice supplier
* Requesting practice name 
* Requesting practice ODS
* Requesting practice supplier 
* Conversation ID
* Timestamp 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import paths
from data.practice_metadata import read_asid_metadata

In [2]:
asid_lookup=read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v2/2021/10/organisationMetadata.json")

transfer_file_location = "s3://prm-gp2gp-transfer-data-preprod/v5/2021/"

transfer_files = [
    "9/2021-9-transfers.parquet",
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]

transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers['month']=transfers['date_requested'].dt.to_period('M')

# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    "SystmOne" : "TPP",
    None: "Unknown"
}

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [3]:
# filter for Vision as the sender with failure reason - Core extract not sent
vision_sender_bool = transfers["sending_supplier"]=="Vision"
core_extract_not_sent_bool = transfers["failure_reason"]=="Core extract not sent"
vision_sender_where_core_extract_not_sent = transfers[vision_sender_bool & core_extract_not_sent_bool].copy()

In [4]:
sorted_vision_sender_where_core_extract_not_sent = vision_sender_where_core_extract_not_sent.sort_values(by="date_requested", ascending=False).reset_index()
columns_to_keep = ['conversation_id', 'date_requested', "requesting_practice_ods_code", "requesting_practice_name", "sending_practice_ods_code", "sending_practice_name", 'requesting_supplier', 'sending_supplier']
sorted_vision_sender_where_core_extract_not_sent = sorted_vision_sender_where_core_extract_not_sent[columns_to_keep]
sorted_vision_sender_where_core_extract_not_sent

,conversation_id,date_requested,requesting_practice_ods_code,requesting_practice_name,sending_practice_ods_code,sending_practice_name,requesting_supplier,sending_supplier
0,9774B1EF-365F-4C36-951E-1BD07A115523,2021-09-30 18:36:25.733,N82117,BROWNLOW GROUP PRACTICE,P83012,TOWER FAMILY HEALTHCARE,EMIS,Vision
1,CB929387-A1A8-4786-9D32-68410B3FC14D,2021-09-30 17:03:28.366,P83001,FAIRFAX GROUP PRACTICE,P83012,TOWER FAMILY HEALTHCARE,Vision,Vision
2,904D2825-2D1D-4199-B813-1C1406CD1AF0,2021-09-30 16:34:52.957,B86003,DR G LEES & PARTNERS,C85024,HIGH STREET PRACTICE,EMIS,Vision
3,A528428B-42B6-432E-9638-B775EB7884E9,2021-09-30 16:07:46.217,N82641,SANDRINGHAM MEDICAL CENTRE,P83012,TOWER FAMILY HEALTHCARE,EMIS,Vision
4,3DDF1550-21FF-11EC-BB2B-6D7F17522011,2021-09-30 15:01:10.845,C83013,RUSKINGTON SURGERY,M89019,HOBS MOAT MEDICAL CENTRE,TPP,Vision
...,...,...,...,...,...,...,...,...
550,69BC9476-EEBA-4A2F-B81A-0412235A1023,2021-09-01 10:39:20.315,P83605,WHITTAKER LANE MED CENTRE,P83001,FAIRFAX GROUP PRACTICE,EMIS,Vision
551,C658007F-1946-4134-85A1-71294D7956EA,2021-09-01 10:20:54.491,L81007,BRIDGE VIEW MEDICAL,G83058,PRIMECARE PMS (SOUTH STREET),EMIS,Vision
552,5ED63279-880B-4656-B84A-BF9B81C630A3,2021-09-01 10:17:28.975,P83612,MILE LANE HEALTH CENTRE,P83608,THE ELMS MEDICAL CENTRE,Vision,Vision
553,1968B7D4-5F8D-4B0C-9B60-B6886D6EF46F,2021-09-01 09:13:10.605,P83605,WHITTAKER LANE MED CENTRE,P83011,UNSWORTH MEDICAL CENTRE,EMIS,Vision


In [6]:
with pd.ExcelWriter("Vision 'Core extract not sent' transfers September 2021 PRMT-2416.xlsx") as writer:
    sorted_vision_sender_where_core_extract_not_sent.to_excel(writer, sheet_name="All",index=False)